In [3]:
import ee
import geemap
import geedim as gd
import geopandas as gpd

In [4]:
geemap.set_proxy(port=4780)

In [7]:
ee.Initialize()

In [8]:
sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

In [28]:
geometry_dir = "/Users/lilywang/Desktop/phD/Qilian/UAV-LULC/downloaded-datasets/Suli/suli.shp"

In [29]:
region = geemap.shp_to_ee(geometry_dir)

In [30]:
def clip_to_shapefile(img):
    return img.clip(region.geometry())

In [99]:
collection = sentinel.filterDate('2021-01-01','2021-12-31').filterBounds(region).map(clip_to_shapefile)


In [40]:
collection.size().getInfo()

368

In [35]:
img = collection.median()

In [39]:
m = geemap.Map()
m.addLayer(img)
m

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [42]:
def calculate_ndvi(img):
    NDVI = img.normalizedDifference(['B8','B4'])
    return NDVI

In [100]:
suli_ndvi = collection.map(calculate_ndvi)

In [49]:
img = suli_ndvi.median()

In [58]:
palette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301']
vis_params = {
    'min': -1,
    'max': 1,
    'palette': [
        'FFFFFF',
        'CE7E45',
        'DF923D',
        'F1B555',
        'FCD163',
        '99B718',
        '74A901',
        '66A000',
        '529400',
        '3E8601',
        '207401',
        '056201',
        '004C00',
        '023B01',
        '012E01',
        '011D01',
        '011301',
    ],
}


In [57]:
suli_ndvi.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'nd',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -1,
    'max': 1},
   'dimensions': [7163, 9084],
   'origin': [3817, 0],
   'crs': 'EPSG:32647',
   'crs_transform': [10, 0, 300000, 0, -10, 4300020]}],
 'properties': {'system:footprint': {'type': 'Polygon',
   'coordinates': [[[97.96039847883067, 38.84446125845723],
     [97.96038392899219, 38.84446185601059],
     [97.16393493888059, 38.834649953238916],
     [97.16366300000004, 38.833895000000034],
     [97.1631510000001, 38.83335100000005],
     [97.16264400000007, 38.83281900000003],
     [97.16121300000003, 38.831316000000065],
     [97.15746000000007, 38.828828000000044],
     [97.15540400000005, 38.82752900000003],
     [97.15370600000006, 38.82582200000007],
     [97.15163500000006, 38.82296500000007],
     [97.15001000000007, 38.820745000000045],
     [97.14711000000005, 38.81805700000007],
     [97.14042300000006, 38.813574000000074],
     [97.13712400000009

In [101]:
suli_ndvi_ts = suli_ndvi.toBands()
ndvi_vis = {
    'min': 0.0,
    'max': 9000.0,
    'palette': 'ndvi',
}
suli_ndvi_ts.getInfo()

{'type': 'Image',
 'bands': [{'id': '20210103T042149_20210103T042426_T47SLC_nd',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -1,
    'max': 1},
   'dimensions': [7163, 9084],
   'origin': [3817, 0],
   'crs': 'EPSG:32647',
   'crs_transform': [10, 0, 300000, 0, -10, 4300020]},
  {'id': '20210103T042149_20210103T042426_T47SLD_nd',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -1,
    'max': 1},
   'dimensions': [7163, 4999],
   'origin': [3817, 5981],
   'crs': 'EPSG:32647',
   'crs_transform': [10, 0, 300000, 0, -10, 4400040]},
  {'id': '20210103T042149_20210103T042426_T47SMC_nd',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -1,
    'max': 1},
   'dimensions': [9874, 9084],
   'crs': 'EPSG:32647',
   'crs_transform': [10, 0, 399960, 0, -10, 4300020]},
  {'id': '20210103T042149_20210103T042426_T47SMD_nd',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -1,
    'max': 1},


In [103]:
Map = geemap.Map()
Map.addLayer(suli_ndvi_ts,{}, 'MODIS NDVI Time-series')
Map.addLayer(suli_ndvi_ts.select(0), ndvi_vis, 'First image')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [104]:
Map.set_plot_options(add_marker_cluster=True)
Map.roi_reducer = ee.Reducer.mean()
Map

Map(bottom=25448.0, center=[38.565347844885466, 97.86074224661395], controls=(WidgetControl(options=['position…

In [80]:
Map.extract_values_to_points('/Users/lilywang/Desktop/phD/Qilian/UAV-LULC/downloaded-datasets/ndvi.csv')

In [105]:
roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(97.1, 38.4, 98.2, 39.2)
    Map.addLayer(roi)
    Map.centerObject(roi)

In [106]:
timelapse = geemap.sentinel1_timelapse(
    roi,
    out_gif='/Users/lilywang/Desktop/phD/Qilian/UAV-LULC/maps/suli_ndvi_ts.gif',
    start_year=2021,
    end_year=2021,
    start_date='01-01',
    end_date='12-31',
    frequency='day',
    vis_params={"min": -1, "max": 1},
    palette=palette,
    frames_per_second=1,
    title='Suli NDVI Timelapse',
    add_colorbar=True,
    colorbar_bg_color='white',
)

Generating URL...
Please wait ...
The GIF image has been saved to: /Users/lilywang/Desktop/phD/Qilian/UAV-LULC/maps/suli_ndvi_ts.gif
ffmpeg is not installed on your computer.
